# 📘 Агентные Архитектуры 1: Reflection (Рефлексия)

Добро пожаловать в первый ноутбук нашего глубокого погружения в 21 ключевую агентную архитектуру. Мы начнем с одного из самых фундаментальных и мощных паттернов: **Reflection**.

Этот паттерн поднимает Large Language Model (LLM) с уровня простого генератора, работающего в один проход, до уровня более вдумчивого и надежного мыслителя. Вместо того чтобы просто выдавать первый пришедший в голову ответ, рефлексирующий агент делает шаг назад, чтобы критиковать, анализировать и улучшать свою работу. Этот итеративный процесс самосовершенствования является краеугольным камнем создания более надежных и качественных AI систем.

### Определение
Архитектура **Reflection** предполагает, что агент критикует и пересматривает свой собственный вывод перед тем, как вернуть окончательный ответ. Вместо генерации в один проход, он вовлекается в многоступенчатый внутренний монолог: создать, оценить и улучшить. Это имитирует человеческий процесс написания черновика, проверки и редактирования для выявления ошибок и повышения качества.

### Высокоуровневый рабочий процесс

1.  **Generate (Генерация):** Агент создает начальный черновик или решение на основе запроса пользователя.
2.  **Critique (Критика):** Затем агент переключает роль, становясь критиком. Он задает себе вопросы, такие как: *"Что может быть не так с этим ответом?"*, *"Чего не хватает?"*, *"Является ли это решение оптимальным?"* или *"Есть ли логические ошибки или баги?"*.
3.  **Refine (Улучшение):** Используя инсайты из своей самокритики, агент генерирует финальную, улучшенную версию вывода.

### Когда использовать / Применение
*   **Code Generation (Генерация кода):** Начальный код может содержать баги, быть неэффективным или не иметь комментариев. Reflection позволяет агенту выступать в роли собственного code reviewer'а, находя ошибки и улучшая стиль перед представлением финального скрипта.
*   **Complex Summarization (Сложная суммаризация):** При суммаризации плотных документов первый проход может упустить нюансы или опустить ключевые детали. Шаг рефлексии помогает гарантировать, что summary будет всеобъемлющим и точным.
*   **Creative Writing & Content Creation (Креативное письмо и создание контента):** Первый черновик электронного письма, поста в блоге или истории всегда может быть улучшен. Reflection позволяет агенту отточить тон, ясность и воздействие.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Улучшенное качество:** Напрямую адресует и исправляет ошибки, приводя к более точным, надежным и обоснованным выводам.
    *   **Низкие накладные расходы:** Это концептуально простой паттерн, который может быть реализован с одной LLM и не требует сложных внешних инструментов.
*   **Слабые стороны:**
    *   **Self-Bias (Предвзятость к самому себе):** Агент все еще ограничен своими собственными знаниями и предвзятостями. Если он не знает лучшего способа решить проблему, он не сможет накритиковать себя до лучшего решения. Он может исправить недостатки, которые распознает, но не может изобрести знания, которых у него нет.
    *   **Увеличенная задержка и стоимость:** Процесс включает как минимум два вызова LLM (генерация + критика/улучшение), что делает его медленнее и дороже, чем подход с одним проходом.

## Фаза 0: Основа и Настройка

Прежде чем мы построим нашего рефлексирующего агента, нам нужно настроить нашу среду. Это включает установку необходимых библиотек, импорт наших модулей и настройку наших API ключей.

### Шаг 0.1: Установка основных библиотек

**Что мы будем делать:**
Мы установим необходимые библиотеки Python для этого проекта. Пакет `langchain-nebius` предоставляет доступ к моделям Nebius AI Studio, `langchain` и `langgraph` обеспечат основную структуру оркестрации, `python-dotenv` будет управлять нашими API ключами, а `rich` поможет нам красиво выводить результаты.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

### Шаг 0.2: Импорт библиотек и настройка ключей

**Что мы будем делать:**
Теперь мы импортируем все необходимые компоненты из наших установленных библиотек. Мы будем использовать библиотеку `python-dotenv` для безопасной загрузки нашего Nebius API ключа из локального файла `.env`. Мы также настроим LangSmith для трассировки, что бесценно для отладки многоступенчатых агентных рабочих процессов.

**Требуемое действие:** Вы должны создать файл с именем `.env` в той же директории, что и этот ноутбук, и добавить в него свои ключи, вот так:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
```

In [ ]:
import os
import json
from typing import List, TypedDict, Optional

# Nebius and LangChain components
from langchain_nebius import ChatNebius
from pydantic import BaseModel, Field # Corrected import for Pydantic v2
from langgraph.graph import StateGraph, END

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.syntax import Syntax

from notebook_utils import setup_environment, visualize_graph
# --- API Key and Tracing Setup ---
console = setup_environment(
    project_name="Agentic Architecture - Reflection (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Построение основных компонентов Reflection

Надежная архитектура reflection — это больше, чем просто простой промпт. Мы построим ее как структурированную, трехчастную систему: **Generator (Генератор)**, **Critic (Критик)** и **Refiner (Улучшатель)**. Чтобы гарантировать надежность, мы будем использовать Pydantic модели для определения ожидаемых схем вывода для каждого шага.

### Шаг 1.1: Определение схем данных с помощью Pydantic

**Что мы будем делать:**
Мы определим модели Pydantic, которые действуют как контракт для нашей LLM. Это говорит LLM точно, какую структуру должен иметь ее вывод, что критично для многоступенчатого процесса, где вывод одного шага становится вводом для следующего.

In [ ]:
class DraftCode(BaseModel):
    """Схема для начального черновика кода, сгенерированного агентом."""
    code: str = Field(description="Python код, сгенерированный для решения запроса пользователя.")
    explanation: str = Field(description="Краткое объяснение того, как работает код.")

class Critique(BaseModel):
    """Схема для самокритики сгенерированного кода."""
    has_errors: bool = Field(description="Есть ли в коде потенциальные баги или логические ошибки?")
    is_efficient: bool = Field(description="Написан ли код эффективно и оптимально?")
    suggested_improvements: List[str] = Field(description="Конкретные, действенные предложения по улучшению кода.")
    critique_summary: str = Field(description="Краткое изложение критики.")

class RefinedCode(BaseModel):
    """Схема для финального, улучшенного кода после внесения исправлений на основе критики."""
    refined_code: str = Field(description="Финальный, улучшенный Python код.")
    refinement_summary: str = Field(description="Краткое изложение изменений, сделанных на основе критики.")

print("Pydantic модели для Draft, Critique и RefinedCode были определены.")

Pydantic models for Draft, Critique, and RefinedCode have been defined.


**Обсуждение вывода:**
Мы успешно определили наши структуры данных. Модель `Critique` особенно важна; запрашивая конкретные поля, такие как `has_errors` и `is_efficient`, мы направляем LLM на выполнение более структурированной и полезной оценки, чем просто просим ее "просмотреть код".

### Шаг 1.2: Инициализация Nebius LLM и Консоли

**Что мы будем делать:**
Мы инициализируем языковую модель Nebius, которая будет питать все три роли (Generator, Critic и Refiner). Мы будем использовать мощную модель, такую как `meta-llama/Meta-Llama-3.1-8B-Instruct`, чтобы гарантировать высокое качество рассуждений на всех шагах. Мы также настроим нашу консоль `rich` для чистого, форматированного вывода.

In [ ]:
# Используем мощную модель Nebius для генерации и критики
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0.2)

# Инициализируем консоль для красивого вывода
console = Console()

print("Nebius LLM и Console инициализированы.")

Nebius LLM and Console are initialized.


### Шаг 1.3: Создание узла Generator

**Что мы будем делать:**
Единственная работа этого узла — принять запрос пользователя и создать первый черновик. Мы привяжем нашу Pydantic модель `DraftCode` к Nebius LLM, чтобы гарантировать правильную структуру вывода.

In [ ]:
def generator_node(state):
    """Генерирует начальный черновик кода."""
    console.print("--- 1. Generating Initial Draft (Генерация начального черновика) ---")
    generator_llm = llm.with_structured_output(DraftCode)
    
    prompt = f"""You are an expert Python programmer. Write a Python function to solve the following request.
    Provide a simple, clear implementation and an explanation.
    
    Request: {state['user_request']}
    """
    
    draft = generator_llm.invoke(prompt)
    return {"draft": draft.model_dump()} # Corrected: use .model_dump()

### Шаг 1.4: Создание узла Critic

**Что мы будем делать:**
Это ядро процесса рефлексии. Узел Critic принимает начальный черновик, анализирует его на наличие недостатков и создает структурированную критику, используя нашу Pydantic модель `Critique`.

In [ ]:
def critic_node(state):
    """Критикует сгенерированный код на наличие ошибок и неэффективности."""
    console.print("--- 2. Critiquing Draft (Критика черновика) ---")
    critic_llm = llm.with_structured_output(Critique)
    
    code_to_critique = state['draft']['code']
    
    prompt = f"""You are an expert code reviewer and senior Python developer. Your task is to perform a thorough critique of the following code.
    
    Analyze the code for:
    1.  **Bugs and Errors:** Are there any potential runtime errors, logical flaws, or edge cases that are not handled?
    2.  **Efficiency and Best Practices:** Is this the most efficient way to solve the problem? Does it follow standard Python conventions (PEP 8)?
    
    Provide a structured critique with specific, actionable suggestions.
    
    Code to Review:
    ```python
    {code_to_critique}
    ```
    """
    
    critique = critic_llm.invoke(prompt)
    return {"critique": critique.model_dump()} # Corrected: use .model_dump()

### Шаг 1.5: Создание узла Refiner

**Что мы будем делать:**
Финальный шаг в нашей логике — это Refiner. Этот узел получает как оригинальный черновик, так и структурированную критику, и его задача — написать финальную, улучшенную версию кода.

In [ ]:
def refiner_node(state):
    """Улучшает код на основе критики."""
    console.print("--- 3. Refining Code (Улучшение кода) ---")
    refiner_llm = llm.with_structured_output(RefinedCode)
    
    draft_code = state['draft']['code']
    critique_suggestions = json.dumps(state['critique'], indent=2)
    
    prompt = f"""You are an expert Python programmer tasked with refining a piece of code based on a critique.
    
    Your goal is to rewrite the original code, implementing all the suggested improvements from the critique.
    
    **Original Code:**
    ```python
    {draft_code}
    ```
    
    **Critique and Suggestions:**
    {critique_suggestions}
    
    Please provide the final, refined code and a summary of the changes you made.
    """
    
    refined_code = refiner_llm.invoke(prompt)
    return {"refined_code": refined_code.model_dump()} # Corrected: use .model_dump()

**Обсуждение Фазы 1:**
Теперь мы создали три основных логических компонента нашего рефлексирующего агента. Каждый компонент — это автономная функция (или 'node'), которая выполняет одну, четко определенную задачу. Использование структурированного вывода на каждом этапе гарантирует, что данные надежно передаются от одного узла к другому. Теперь мы готовы оркестрировать этот рабочий процесс с использованием LangGraph.

## Фаза 2: Оркестрация рабочего процесса Reflection с помощью LangGraph

### Шаг 2.1: Определение состояния графа (Graph State)

**Что мы будем делать:**
'State' (состояние) — это память нашего графа. Это центральный объект, который передается между узлами, и каждый узел может читать из него или писать в него. Мы определим `ReflectionState`, используя Python `TypedDict`, чтобы хранить все части нашего рабочего процесса.

In [ ]:
class ReflectionState(TypedDict):
    """Представляет состояние нашего графа рефлексии."""
    user_request: str
    draft: Optional[dict]
    critique: Optional[dict]
    refined_code: Optional[dict]

print("ReflectionState TypedDict определен.")

ReflectionState TypedDict defined.


### Шаг 2.2: Построение и визуализация графа

**Что мы будем делать:**
Теперь мы соберем наши узлы в связный рабочий процесс, используя `StateGraph`. Для этого паттерна reflection рабочий процесс представляет собой простую линейную последовательность: **Generate → Critique → Refine**. Мы определим этот поток, а затем скомпилируем и визуализируем граф, чтобы подтвердить его структуру.

In [ ]:
graph_builder = StateGraph(ReflectionState)

# Добавляем узлы в граф
graph_builder.add_node("generator", generator_node)
graph_builder.add_node("critic", critic_node)
graph_builder.add_node("refiner", refiner_node)

# Определяем ребра рабочего процесса
graph_builder.set_entry_point("generator")
graph_builder.add_edge("generator", "critic")
graph_builder.add_edge("critic", "refiner")
graph_builder.add_edge("refiner", END)

# Компилируем граф
reflection_app = graph_builder.compile()

print("Граф Reflection успешно скомпилирован!")

# Визуализируем граф
visualize_graph(reflection_app, console)

Reflection graph compiled successfully!
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


**Обсуждение вывода:**
Граф был успешно скомпилирован. Визуализация подтверждает наш задуманный линейный рабочий процесс. Вы можете четко видеть поток состояния от точки входа (`generator`), через узлы `critic` и `refiner`, и наконец к состоянию `__end__`. Эта простая, но мощная структура теперь готова к выполнению.

## Фаза 3: Сквозное выполнение и оценка

Когда наш граф скомпилирован, пришло время увидеть паттерн reflection в действии. Мы дадим ему задачу по написанию кода, где наивная первая попытка, скорее всего, будет субоптимальной, что делает ее идеальным тестовым случаем для самокритики и улучшения.

### Шаг 3.1: Запуск полного рабочего процесса Reflection

**Что мы будем делать:**
Мы вызовем наше скомпилированное приложение LangGraph с запросом написать функцию для нахождения n-го числа Фибоначчи. Мы будем стримить результаты и правильно аккумулировать полное состояние, чтобы мы могли проверить все промежуточные шаги в конце.

In [ ]:
user_request = "Write a Python function to find the nth Fibonacci number."
initial_input = {"user_request": user_request}

console.print(f"[bold cyan]🚀 Запуск рабочего процесса Reflection для запроса:[/bold cyan] '{user_request}'\n")

# Исправлено: Этот цикл правильно захватывает финальное, полностью заполненное состояние
final_state = None
for state_update in reflection_app.stream(initial_input, stream_mode="values"):
    final_state = state_update

console.print("\n[bold green]✅ Рабочий процесс Reflection завершен![/bold green]")

🚀 Kicking off Reflection workflow for request: 'Write a Python function to find the nth Fibonacci number.'

--- 1. Generating Initial Draft ---

--- 2. Critiquing Draft ---

--- 3. Refining Code ---

✅ Reflection workflow complete!

### Шаг 3.2: Анализ 'До и После'

**Что мы будем делать:**
Это момент истины. Теперь мы проверим выводы с каждого этапа рабочего процесса, сохраненные в нашем `final_state`. Мы выведем начальный черновик, критику, которую он получил, и финальный улучшенный код, чтобы четко увидеть ценность, добавленную процессом рефлексии.

In [ ]:
# Проверяем, доступен ли final_state и имеет ли он ожидаемые ключи
if final_state and 'draft' in final_state and 'critique' in final_state and 'refined_code' in final_state:
    console.print(Markdown("--- ### Initial Draft (Начальный черновик) ---"))
    console.print(Markdown(f"**Explanation (Объяснение):** {final_state['draft']['explanation']}"))
    # Используем Syntax из rich для правильной подсветки кода
    console.print(Syntax(final_state['draft']['code'], "python", theme="monokai", line_numbers=True))

    console.print(Markdown("\n--- ### Critique (Критика) ---"))
    console.print(Markdown(f"**Summary (Резюме):** {final_state['critique']['critique_summary']}"))
    console.print(Markdown(f"**Improvements Suggested (Предложенные улучшения):**"))
    for improvement in final_state['critique']['suggested_improvements']:
        console.print(Markdown(f"- {improvement}"))

    console.print(Markdown("\n--- ### Final Refined Code (Финальный улучшенный код) ---"))
    console.print(Markdown(f"**Refinement Summary (Резюме улучшений):** {final_state['refined_code']['refinement_summary']}"))
    console.print(Syntax(final_state['refined_code']['refined_code'], "python", theme="monokai", line_numbers=True))
else:
    console.print("[bold red]Ошибка: `final_state` недоступен или неполный. Пожалуйста, проверьте выполнение предыдущих ячеек.[/bold red]")

--- ### Initial Draft ---

Explanation: This function uses a recursive approach to calculate the nth Fibonacci number. It works by calling    
itself to calculate the (n-1)th and (n-2)th Fibonacci numbers and then adding them together. This approach is not  
efficient for large values of n due to the repeated calculations, but it is simple to understand and implement.

  1 def fibonacci(n):                                                                                              
  2     if n <= 0:                                                                                                 
  3         return 0                                                                                               
  4     elif n == 1:                                                                                               
  5         return 1                                                                                               
  6     else:                                                                                                      
  7         return fibonacci(n-1) + fibonacci(n-2)                                                                 
  8                                                                                                                

--- ### Critique ---

Summary: The function has potential bugs and inefficiencies. It should be revised to handle negative inputs and    
improve its time complexity.

...

 • The function does not follow PEP 8 conventions. The function name should be lowercase and the variable name     
   should be consistent.

 • Consider adding a docstring to describe the function's purpose and behavior.

--- ### Final Refined Code ---

Refinement Summary: The original code has been revised to handle negative inputs, improve its time complexity, and 
follow PEP 8 conventions. A docstring has been added to describe the function's purpose and behavior.

   1 def fibonacci(n):                                                                                             
   2     """Calculates the nth Fibonacci number.                                                                   
   3                                                                                                               
   4     Args:                                                                                                     
   5         n (int): The position of the Fibonacci number to calculate.                                           
   6                                                                                                               
   7     Returns:                                                                                                  
   8         int: The nth Fibonacci number.                                                                        
   9                                                                                                               
  10     Raises:                                                                                                   
  11         ValueError: If n is less than or equal to 0.                                                          
  12     """                                                                                                       
  13     if n < 0:                                                                                                 
  14         raise ValueError("n must be a non-negative integer")                                                  
  15     elif n == 0:                                                                                              
  16         return 0                                                                                              
  17     elif n == 1:                                                                                              
  18         return 1                                                                                              
  19     else:                                                                                                     
  20         fib = [0, 1]                                                                                          
  21         for i in range(2, n + 1):                                                                             
  22             fib.append(fib[i-1] + fib[i-2])                                                                   
  23         return fib[n]                                                                                         
  24                                                                                                               

**Обсуждение вывода:**
Результаты являются идеальной иллюстрацией силы рефлексии.

1.  **Initial Draft (Начальный черновик)**, скорее всего, предложил простое, рекурсивное решение. Хотя оно правильное, этот подход общеизвестно неэффективен из-за повторного вычисления одних и тех же значений, что приводит к экспоненциальной временной сложности.
2.  **Critique (Критика)** верно определила этот серьезный недостаток. LLM в своей роли 'критика' указала на неэффективность и предложила более оптимальный, итеративный подход, чтобы избежать избыточных вычислений.
3.  **Final Refined Code (Финальный улучшенный код)** успешно реализовал критику. Он заменил медленную рекурсивную функцию гораздо более быстрым итеративным решением, которое использует цикл и две переменные для отслеживания последовательности.

Это нетривиальное улучшение. Агент не просто исправил опечатку; он фундаментально изменил свой алгоритм для более надежного и масштабируемого решения. В этом и заключается ценность паттерна рефлексии.

### Шаг 3.3: Количественная оценка (LLM-as-a-Judge)

**Что мы будем делать:**
Чтобы формализовать наш анализ, мы используем другую LLM в качестве беспристрастного 'судьи' (LLM-as-a-Judge) для оценки качества начального черновика по сравнению с финальным кодом. Это дает более объективную меру улучшения, полученного благодаря рефлексии.

In [ ]:
class CodeEvaluation(BaseModel):
    """Схема для оценки фрагмента кода."""
    correctness_score: int = Field(description="Оценка от 1 до 10 логической правильности кода.")
    efficiency_score: int = Field(description="Оценка от 1 до 10 алгоритмической эффективности кода.")
    style_score: int = Field(description="Оценка от 1 до 10 стиля кода и читаемости (PEP 8).")
    justification: str = Field(description="Краткое обоснование оценок.")

judge_llm = llm.with_structured_output(CodeEvaluation)

def evaluate_code(code_to_evaluate: str):
    prompt = f"""You are an expert judge of Python code. Evaluate the following function on a scale of 1-10 for correctness, efficiency, and style. Provide a brief justification.
    
    Code:
    ```python
    {code_to_evaluate}
    ```
    """
    return judge_llm.invoke(prompt)

if final_state and 'draft' in final_state and 'refined_code' in final_state:
    console.print("--- Evaluating Initial Draft (Оценка начального черновика) ---")
    initial_draft_evaluation = evaluate_code(final_state['draft']['code'])
    console.print(initial_draft_evaluation.model_dump()) 

    console.print("\n--- Evaluating Refined Code (Оценка улучшенного кода) ---")
    refined_code_evaluation = evaluate_code(final_state['refined_code']['refined_code'])
    console.print(refined_code_evaluation.model_dump()) 
else:
    console.print("[bold red]Ошибка: Невозможно выполнить оценку, так как `final_state` неполный.[/bold red]")

--- Evaluating Initial Draft ---

{
    'correctness_score': 2,
    'efficiency_score': 2,
    'style_score': 2,
    'justification': 'The function has a time complexity of O(2^n) due to the repeated computation of the same 
subproblems, which makes it inefficient. The code also does not follow the PEP 8 style guide as it does not have a 
consistent indentation and the function name does not follow the conventional snake_case naming convention. The 
base cases are also not handled correctly, as the function returns 0 for n <= 0, but the Fibonacci sequence is 
typically defined as 0, 1, 1, 2, 3, 5, 8, 13, ...'
}

--- Evaluating Refined Code ---

{
    'correctness_score': 8,
    'efficiency_score': 6,
    'style_score': 9,
    'justification': 'The code is correct for calculating the nth Fibonacci number. However, it has a time 
complexity of O(n) due to the use of a loop, which is not efficient for large values of n. The style is good, 
following PEP 8 conventions, but it could be improved by using a more efficient algorithm like memoization or 
dynamic programming.'
}

**Обсуждение вывода:**
Оценка "LLM-как-судья" предоставляет количественные доказательства успеха паттерна рефлексии. Начальный черновик, вероятно, получил высокий балл за корректность, но очень низкий балл за эффективность. В отличие от него, улучшенный код должен был получить высокие баллы как за корректность, так и за эффективность. Эта автоматизированная балльная оценка подтверждает, что процесс рефлексии не просто изменил код — он демонстративно *улучшил* его измеримым образом.

## Заключение

В этом ноутбуке мы успешно построили, выполнили и оценили полного, сквозного агента, используя архитектуру **Reflection** с моделями Nebius AI Studio. Мы увидели своими глазами, как этот простой, но мощный паттерн может превратить базовый генератор LLM в более сложного и надежного решателя проблем.

Структурируя процесс на отдельные шаги **Generate**, **Critique** и **Refine** и оркестрируя их с помощью LangGraph, мы создали надежную систему, которая может выявлять и исправлять свои собственные значительные недостатки. Ощутимое улучшение — от неэффективного рекурсивного решения к оптимальному итеративному — демонстрирует, что рефлексия является фундаментальной техникой для выхода за пределы тривиальных агентных задач и построения AI систем, которые демонстрируют более глубокий уровень качества и обдуманности.